In [ ]:
import bs4 as bs
import cv2
import csv
import glob
import pytesseract
import os
import urllib.request
import httplib2
from inaFaceAnalyzer.inaFaceAnalyzer import ImageAnalyzer

from PIL import Image
from sentence_transformers import SentenceTransformer, util
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
